In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
plt.rcParams['figure.figsize'] = 8,8

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
#import keras
import numpy as np
import os, math
from matplotlib.pyplot import figure
#from keras.backend import tensorflow_backend
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.metrics import structural_similarity as ssim
import math
import cv2
import glob
import re
import tensorflow as tf
from skimage import segmentation, color
from skimage.io import imread
from skimage.future import graph
from matplotlib import pyplot as plt


In [2]:
input_img = Input(shape=(None,None,3))
    # 1st layer, Conv+relu
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(input_img)
x = Activation('relu')(x)
    # 15 layers, Conv+BN+relu
for i in range(15):
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    #x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
    x = Activation('relu')(x)   
    # last layer, Conv
x = Conv2D(filters=3, kernel_size=(3,3), strides=(1,1), padding='same')(x)
x = Subtract()([input_img, x])   # input - noise

model = Model(input_img, x)


opt = keras.optimizers.Adam(learning_rate=0.0003)

model.compile(loss='mae', optimizer=opt,    metrics=[  "accuracy"])
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
 

2024-06-23 12:28:42.659267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-23 12:28:42.682325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-23 12:28:42.682447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-23 12:28:42.682972: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
def msse(y_true, y_pred):
        
    return K.mean(tf.keras.losses.mse(y_true, y_pred))


def nod(x,fc):
    nx= np.random.normal(scale=fc/255, size=x.shape)
    ad= np.float32(np.clip(x+nx,0,1))
    return ad

### Following noise adding function is critical, as it aligns with the normalization operation belongs to the given trained model.

In [4]:
def nod(x,fc):
    nx= np.random.normal(scale=fc/255, size=x.shape)
    ad= np.float32(np.clip(x+nx,0,1))
    return ad

In [5]:

def pgy(parameters, gradients ):
    "Used for convex summation"
    new_grads = []
    for (params, grads) in zip(parameters, gradients):
 
        ap = 9e-1
        gt = (1-ap)*   (grads)  +(ap*params)
        new_grads.append( (gt)  )
    return new_grads 




def test_nup1(mofel,test_img,no_lvl):
    "SS-TTA optimization 1"
    t1 = test_img
    g1 = np.expand_dims(test_img,0)
    ng1 = nod(g1,no_lvl)
    ng2 = nod(ng1,25)
    ewt = mofel.get_weights()
    #mo1 = Model(input_img, x)
    mo1 = mofel
    mo1.set_weights(ewt)
    xe1 = mofel.predict(ng1)
    xe2 = nod(xe1,no_lvl)
    
    
    for i in range(5):
    
        with tf.GradientTape(persistent=True) as tape:

            
                to1 = (mo1(xe2, training=True) )
                to2 = (mo1(nod(xe2,5), training=True) )
                to3 =  (mo1(xe1, training=True) )
 
                loj = msse(xe1,to1) + msse(to2,to1) + msse(xe1,to3)
 
                            
    
        grads =   (tape.gradient(loj, mo1.trainable_weights))
 
        optimizer.apply_gradients(zip(grads, mo1.trainable_weights))
        ko1 = mo1.get_weights()
        ko1 = pgy(ewt,ko1)
        mo1.set_weights(ko1) 
    frp = np.clip ( (mo1.predict(ng1))[0],0,1)
        #ng2 = (ng2+to1)/2
    
 
    
    return frp






 



def test_nup2(mofel,test_img,no_lvl):
    "SS-TTA optimization 2"
    t1 = test_img
    g1 = np.expand_dims(test_img,0)
    ng1 = nod(g1,no_lvl)
    ng2 = nod(ng1,no_lvl)
    ewt = mofel.get_weights()
    #mo1 = Model(input_img, x)
    mo1 = mofel
    mo1.set_weights(ewt)
    xe1 = mofel.predict(ng1)
    xe2 = nod(xe1,no_lvl)
    
    
    
    for i in range(5):
    
        with tf.GradientTape(persistent=True) as tape:

            
                to1 = (mo1(xe2, training=True) )
                dh = xe1 -to1
                to2 = (mo1(nod(xe2,5), training=True) )
                to3 =  (mo1(xe1, training=True) )
 
                loj = msse(xe1+dh,to1)  + msse(xe1+dh,to2)  + msse(xe1,to3)
 
                            
    
        grads =   (tape.gradient(loj, mo1.trainable_weights))
 
        optimizer.apply_gradients(zip(grads, mo1.trainable_weights))
        ko1 = mo1.get_weights()
        ko1 = pgy(ewt,ko1)
        mo1.set_weights(ko1) 
    frp = np.clip ( (mo1.predict(ng1))[0],0,1)
        #ng2 = (ng2+to1)/2
    
 
    
    return frp

In [6]:
# from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error 
from glob import glob




optimizer = keras.optimizers.Adam(learning_rate=0.0003)

vo1 = list()
vo1.append('5set')
vo1.append('14set')
vo1.append('bsdp')
vo1.append('kodak')
vo1.append('ur100d')
vo1 = np.array(vo1)


id1 = 4 #dataset

ar=[]
gsigma= 50



aq = vo1[id1]
    
    
datat = np.array(glob('./datas/'+ aq+ '/*'))


s1=list()
s2=list()
s3=list()


for co in range (len(datat )):
    ar= cv2.imread(datat[co])
    hf=512
 
    a= cv2.cvtColor(ar, cv2.COLOR_BGR2RGB)/255
 
 
    img = np.asarray(a) 
    
    pr = nod(np.expand_dims(img,axis=0), gsigma)
    
    
    
    model.load_weights('./A_ns2ns_25.h5') 
    
    prd_bs =  model.predict(pr)
    prd_bs  = np.clip(prd_bs ,0,1)
    
    
    
    
    model.load_weights('./A_ns2ns_25.h5') 
    
    prd_tu1   =  test_nup1(model,img,  gsigma)
    prd_tu1   =  np.clip(prd_tu1  ,0,1)

    
    
    model.load_weights('./A_ns2ns_25.h5') 
    
    prd_tu2 =  test_nup2(model,img,  gsigma)
    
    prd_tu2  = np.clip(prd_tu2 ,0,1)
 



 
    psnr_const0 = tf.image.psnr(a, prd_bs ,max_val=1).numpy()
    psnr_const1 = tf.image.psnr(a, prd_tu1 ,max_val=1).numpy()
    psnr_const2 = tf.image.psnr(a, prd_tu2 ,max_val=1).numpy()
    
    
    print(co,'psnr',psnr_const0,psnr_const1,psnr_const2)
 
    
    
    s1.append(psnr_const0)
    s2.append(psnr_const1)
    s3.append(psnr_const2)
 

ms1=np.asarray(s1)
ms2=np.asarray(s2)
ms3=np.asarray(s3)

print('mean_psnr:- ' 'baseline:',np.mean(ms1), '||opt 1:',np.mean(ms2), '||opt 2:',np.mean(ms3))

 

 


2024-06-23 12:28:43.936083: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8801


0 psnr [26.190414] 26.282475 26.36333
1 psnr [24.694891] 24.754818 24.771399
2 psnr [26.516754] 26.663948 26.781193
3 psnr [25.660297] 25.708302 25.79735
4 psnr [27.853823] 27.975246 28.004667
5 psnr [27.035364] 27.223534 27.252537
6 psnr [25.66413] 25.868084 26.090988
7 psnr [26.387138] 26.52251 26.597689
8 psnr [29.6664] 29.780891 29.961205
9 psnr [24.872116] 25.02835 25.32527
10 psnr [23.37018] 23.473705 23.606604
11 psnr [25.654001] 25.884005 26.193865
12 psnr [26.098068] 26.337236 26.489803
13 psnr [22.423306] 22.418913 22.468443
14 psnr [27.863937] 27.929901 28.012539
15 psnr [27.52017] 27.65437 27.784328
16 psnr [25.941172] 25.95948 26.082436
17 psnr [26.237309] 26.248362 26.391247
18 psnr [25.7087] 25.742878 25.78168
19 psnr [25.635094] 25.638746 25.775637
20 psnr [26.739956] 26.959538 27.409628
21 psnr [29.023806] 29.114662 29.202038
22 psnr [23.365837] 23.32976 23.405313
23 psnr [25.855005] 25.881088 25.924318
24 psnr [27.361727] 27.376232 27.450342
25 psnr [26.486382] 26.596